# Collecting Typical Blobs based on pre-detected cells

In [43]:
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import sys, os, time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'cells'))
# sys.path.append(os.path.join(os.environ['REPO_DIR'], 'annotation'))
# sys.path.append(os.path.join(os.environ['REPO_DIR'], 'learning'))

import numpy as np
import scipy
import skimage
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from skimage.measure import regionprops, label, find_contours
from skimage.color import rgb2hsv
from scipy import ndimage as ndi
import pandas
from scipy.signal import argrelmax
from scipy.stats import linregress

from annotation_utilities import *
from registration_utilities import *
from learning_utilities import *
from data_manager import *
from utilities2015 import *
from cell_utilities import *
from metadata import *
from distributed_utilities import download_from_s3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
save_folder_path = '/shared/blob_matching_atlas/'
from xj_utilities import *
fun_create_folder(save_folder_path=save_folder_path)

Folder already exists: /shared/blob_matching_atlas/

In [35]:
scan_parameters = {}
scan_parameters['stack'] = 'MD589'
scan_parameters['patch_size'] = 448
scan_parameters['patch_half_size'] = scan_parameters['patch_size']/2
scan_parameters['stride'] = 112
scan_parameters['crop_0_min'] = 7000
scan_parameters['crop_1_min'] = 3000
scan_parameters['crop_0_max'] = 13000
scan_parameters['crop_1_max'] = 14000
scan_parameters['im0max'] = scan_parameters['crop_0_max'] - scan_parameters['crop_0_min']
scan_parameters['im1max'] = scan_parameters['crop_1_max'] - scan_parameters['crop_1_min']
scan_parameters['section_limits'] = metadata_cache['section_limits'][scan_parameters['stack']]
scan_parameters['o_crop'] = True
scan_parameters['o_clear_border'] = True
scan_parameters['o_relabel'] = True
scan_parameters['crop_range_mxmx'] =(scan_parameters['crop_0_min'],scan_parameters['crop_0_max'],scan_parameters['crop_1_min'],scan_parameters['crop_1_max'])
scan_parameters['oriImL1'], scan_parameters['oriImL0'] = metadata_cache['image_shape'][scan_parameters['stack']]
scan_parameters['scan_section_range'] = 1

scan_parameters['prop'] = ['centroid','eccentricity','area','orientation','moments_hu','bbox','equivalent_diameter']
scan_parameters['prop_for_comparison'] = ['area', 'orientation','moments_hu']
scan_parameters['compare_weight'] = [1 for i in range(len(scan_parameters['prop_for_comparison']) + 6)]
scan_parameters['compare_weight'] = np.array(scan_parameters['compare_weight'])/ float(np.sum(scan_parameters['compare_weight']))
scan_parameters['similarity_threshold'] = 0.8
scan_parameters['o_fix_scan_size'] = True
scan_parameters['scan_size'] = 112
scan_parameters['scan_size_coeff'] = 5
scan_parameters['prop_to_save'] = ['coords','moments_hu','centroid','area','eccentricity','equivalent_diameter']


In [36]:
patch_size = scan_parameters['patch_size']
patch_half_size = scan_parameters['patch_half_size']
stride = scan_parameters['stride']
# Brain stem
crop_0_min = scan_parameters['crop_0_min']
crop_0_max = scan_parameters['crop_1_min']
crop_1_min = scan_parameters['crop_0_max']
crop_1_max = scan_parameters['crop_1_max']
# Inside
# crop_0_min = 4000
# crop_0_max = 6000
# crop_1_min = 2400
# crop_1_max = 4400
# Whole
# crop_0_min = 0
# crop_1_min = 0
# _, _, crop_1_max, crop_0_max = get_default_gridspec(stack=stack, patch_size=patch_size, stride=stride)
im0max = crop_0_max - crop_0_min
im1max = crop_1_max - crop_1_min

# Options
option_rescale_grayscale = True
option_THimage_save = True;
option_save_image_extension = '.png'

# Process images
option_label_connectivity = 4;

### Load and crop origninal images

In [157]:
# Functionalize loading data
img_filename = {}
stack = scan_parameters['stack']
section_begin, section_end = scan_parameters['section_limits']
secList = range(section_begin, section_end+1)
tempSecList = range(section_begin, section_end+1)
# Download all the data for MD589
for tempSec in tempSecList:
    try:
        img_filename[tempSec] = DataManager.get_image_filepath(stack=stack, section=tempSec, resol='lossless', version='cropped')
    except:
        sys.stderr.write('Invalid section %d...\n'%tempSec)
        secList.remove(tempSec)
        continue
    if os.path.isfile(img_filename[tempSec]):
        sys.stderr.write('Image file is already available. \n')
        continue
    else:
        try:
            download_from_s3(img_filename[tempSec])
        except:
            sys.stderr.write('Download fail. Skip this file for section %d...\n'%tempSec)
            secList.remove(tempSec)
            continue   
scan_parameters['secList'] = secList

Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Invalid section 118...
Child returned 0


rm -rf /shared/CSHL_data_processed/MD589/MD589_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped/MD589-IHC21-2015.07.30-22.40.00_MD589_1_0061_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped.tif && mkdir -p /shared/CSHL_data_processed/MD589/MD589_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped/MD589-IHC21-2015.07.30-22.40.00_MD589_1_0061_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped.tif /shared/CSHL_data_processed/MD589/MD589_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped/MD589-IHC21-2015.07.30-22.40.00_MD589_1_0061_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped.tif


Child returned 0
2.03 seconds.
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is alr

rm -rf /shared/CSHL_data_processed/MD589/MD589_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped/MD589-N42-2015.07.30-18.52.37_MD589_1_0124_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped.tif && mkdir -p /shared/CSHL_data_processed/MD589/MD589_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped/MD589-N42-2015.07.30-18.52.37_MD589_1_0124_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped.tif /shared/CSHL_data_processed/MD589/MD589_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped/MD589-N42-2015.07.30-18.52.37_MD589_1_0124_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped.tif


Child returned 0
2.13 seconds.
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is alr

rm -rf /shared/CSHL_data_processed/MD589/MD589_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped/MD589-IHC58-2015.07.31-01.22.39_MD589_1_0172_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped.tif && mkdir -p /shared/CSHL_data_processed/MD589/MD589_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped/MD589-IHC58-2015.07.31-01.22.39_MD589_1_0172_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped.tif /shared/CSHL_data_processed/MD589/MD589_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped/MD589-IHC58-2015.07.31-01.22.39_MD589_1_0172_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped.tif


Child returned 0
2.08 seconds.
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Invalid section 324...
Child returned 0


rm -rf /shared/CSHL_data_processed/MD589/MD589_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped/MD589-IHC62-2015.07.31-01.39.26_MD589_1_0184_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped.tif && mkdir -p /shared/CSHL_data_processed/MD589/MD589_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped/MD589-IHC62-2015.07.31-01.39.26_MD589_1_0184_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped.tif /shared/CSHL_data_processed/MD589/MD589_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped/MD589-IHC62-2015.07.31-01.39.26_MD589_1_0184_lossless_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped.tif


Child returned 0
2.09 seconds.
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is already available. 
Image file is alr

In [159]:
# typical_blobs = fun_load_data_collect_typical_blobs(sec=sec, scan_parameters=scan_parameters,o_save=False)

In [8]:
from multiprocess import Pool

In [9]:
result = Pool(NUM_CORES-1)
result.map(lambda sec: fun_load_data_collect_typical_blobs(sec=sec, scan_parameters=scan_parameters,o_save=True), secList)

Warrning: Scaned section(s) not included in input im_blob_prop
Section 117 Finished percentage: 0.000000
Section 117 Finished percentage: 14.923146
Section 123 Finished percentage: 0.000000
Section 117 Finished percentage: 29.846292
Section 128 Finished percentage: 0.000000
Warrning: Scaned section(s) not included in input im_blob_prop
Section 92 Finished percentage: 0.000000
Section 112 Finished percentage: 0.000000
Section 133 Finished percentage: 0.000000
Section 143 Finished percentage: 0.000000
Section 92 Finished percentage: 19.451469
Section 117 Finished percentage: 44.769437
Section 138 Finished percentage: 0.000000
Section 153 Finished percentage: 0.000000
Section 158 Finished percentage: 0.000000
Section 102 Finished percentage: 0.000000
Section 97 Finished percentage: 0.000000
Section 123 Finished percentage: 21.753317
Section 128 Finished percentage: 20.508614
Section 107 Finished percentage: 0.000000
Section 92 Finished percentage: 38.902937
Section 148 Finished percentage

your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->[0]]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


Result saved.
rm -rf /shared/CSHL_cells_v2/detected_cells/MD589/MD589-IHC21-2015.07.30-22.40.00_MD589_1_0061/MD589-IHC21-2015.07.30-22.40.00_MD589_1_0061_blobCoords.hdf && mkdir -p /shared/CSHL_cells_v2/detected_cells/MD589/MD589-IHC21-2015.07.30-22.40.00_MD589_1_0061


aws s3 cp s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD589/MD589-IHC21-2015.07.30-22.40.00_MD589_1_0061/MD589-IHC21-2015.07.30-22.40.00_MD589_1_0061_blobCoords.hdf /shared/CSHL_cells_v2/detected_cells/MD589/MD589-IHC21-2015.07.30-22.40.00_MD589_1_0061/MD589-IHC21-2015.07.30-22.40.00_MD589_1_0061_blobCoords.hdf
Section 143 Finished percentage: 11.843430
Section 163 Finished percentage: 4.188833
Section 92 Finished percentage: 77.805874


Child returned 0
1.94 seconds.


...

Child returned 0


aws s3 cp s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD589/MD589-N69-2015.07.30-20.46.59_MD589_3_0207/MD589-N69-2015.07.30-20.46.59_MD589_3_0207_blobCoords.hdf /shared/CSHL_cells_v2/detected_cells/MD589/MD589-N69-2015.07.30-20.46.59_MD589_3_0207/MD589-N69-2015.07.30-20.46.59_MD589_3_0207_blobCoords.hdf


Child returned 0
1.48 seconds.


Section 364 Finished percentage: 25.163563
Section 350 Finished percentage: 48.320850
Section 369 Finished percentage: 0.000000
Section 354 Finished percentage: 36.390102
Section 359 Finished percentage: 0.000000
Section 364 Finished percentage: 50.327126
Section 350 Finished percentage: 72.481276
Section 369 Finished percentage: 31.585597
Section 354 Finished percentage: 54.585153
Section 359 Finished percentage: 23.501763
Section 350 Finished percentage: 96.641701
Result saved.
Section 364 Finished percentage: 75.490689
Section 369 Finished percentage: 63.171194
Section 354 Finished percentage: 72.780204
Section 359 Finished percentage: 47.003525
Result saved.
Section 354 Finished percentage: 90.975255
Section 369 Finished percentage: 94.756791
Result saved.
Result saved.
Section 359 Finished percentage: 70.505288
Section 365 Finished percentage: 0.000000
Warrning: Scaned section(s) not included in input im_blob_prop
Section 370 Finished percentage: 0.000000
Section 370 Finished perc

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [83]:
sec = 152

In [160]:
result = Pool(NUM_CORES-1)
result.map(lambda sec: fun_vis_typical_blob(stack=stack, sec=sec, o_overlay_on_oriImage=True, o_save_image=True), secList)

['Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image saved',
 'Image 

In [161]:
upload_to_s3('/shared/blob_matching_atlas',is_dir=True)

aws s3 cp --recursive /shared/blob_matching_atlas s3://mousebrainatlas-data/blob_matching_atlas


Child returned 0
19.42 seconds.
